# Create SPECFEM3D Project
1. Unpickle events and staions
1. Read and map moment tensors to events
1. Prep observed station data for writing to ascii files
1. Setup and test DATA directories and specfem3d file names
1. Create directories and write CMTSOLUTION, STATION, STATION_ADJOINT, and trace ascii files in separate dictories for each event
1. Pick P-arrivals
1. Write adjoint source window parameter files

#### 1. Unpickle events and stations 

In [ ]:
import pickle
from gnam.events.gevents import gevents as gevents
from gnam.events.gstations import gstations as gstations
from gnam.events.munge.knmi import correct_station_depths as csd_f

#Unpickle events
print('Unpickling Events')
f = open('../../../data_notebooks/pickled/events.pickle', 'rb')
dill_events = pickle.load(f)
f.close()
    

#Unpickle stations
print('Unpickling Station Traces')
f = open('../../../data_notebooks/pickled/straces.pickle', 'rb')
dill_straces = pickle.load(f)
f.close()

# This is kind of hokey, but it works for now.
# Some of the stations depths do not follow the 
# 50, 100, 150, 200 meter depths -- possibly because
# the boreholes are slanted. To correct for this,
# a hard coded "patch/update" is applied. See the
# code for details and update values.
dill_straces.correct_stations(csd_f)

print('Done')

#### 2. Read and map moment tensors to events

In [ ]:
from gnam.events.mtensors import mtensors
import pandas as pd
import numpy as np

#Read moment tensors
gf_mts = mtensors('../../../data_notebooks/data/event_moments.csv')
    
# get event catalog of the events withing the bounding box
e_cat = dill_events.getIncCatalog()

# This is a bit hokey, but it works. Here we update the
# event time from the moment tensor CSV file with thouse
# from the event catalog
gf_mts.update_utcdatetime(e_cat)

# Create a dictionary that maps moment tensors to events
e2mt_dict = gf_mts.map_events_2_tensors(e_cat)
e2mt_keys = e2mt_dict.keys()

# Print a comparison of events to moment tensors
for key in e2mt_keys:
    print('UTC: event[%d][Date] = %s' %(key,e_cat[key].origins[0].time))
    print('UTC:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['Date']))
    print('Mag: event[%d][Date] = %s' %(key,e_cat[key].magnitudes[0].mag))
    print('Mag:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['ML']))
    print()

#### 3. Prep observed station data for writing to ascii files

In [ ]:
from obspy import Stream

#create streams and map eventid to stream
e_stream_dict = {}
ce_stream_dict = {}
for ekey in e2mt_keys:
    '''
    e_st  = Stream()
    e_st += dill_straces.getStreamZ(ekey,3) 
    e_st += dill_straces.getStreamZ(ekey,4) 
    e_st.resample(1000)
    e_stream_dict[ekey] = e_st
    '''
    
    #get boreholes 3 and 4 streams for the event
    e1_st3 = dill_straces.getStream1(ekey,3)
    e1_st4 = dill_straces.getStream1(ekey,4)
    e2_st3 = dill_straces.getStream2(ekey,3)
    e2_st4 = dill_straces.getStream2(ekey,4)
    ez_st3 = dill_straces.getStreamZ(ekey,3)
    ez_st4 = dill_straces.getStreamZ(ekey,4)

    #combine component streams
    e_st3 = e1_st3 + e2_st3 + ez_st3
    e_st4 = e1_st4 + e2_st4 + ez_st4

    #reorder streams so that each station component is contiguous
    te_st3 = Stream()
    te_st4 = Stream()

    ns = len(ez_st3)
    for i in range(ns):
        te_st3 += e_st3[i+2*ns]
        te_st3 += e_st3[i+ns]
        te_st3 += e_st3[i]

        te_st4 += e_st4[i+2*ns]
        te_st4 += e_st4[i+ns]
        te_st4 += e_st4[i]
    
    e_st3 = te_st3
    e_st4 = te_st4
    
    #rotate the streams to ZNE components (obspy and the FDSN info is used)
    inv3 = dill_straces.get_inventory(3)
    inv4 = dill_straces.get_inventory(4)
    e_st3.rotate(method="->ZNE", inventory=inv3)
    e_st4.rotate(method="->ZNE", inventory=inv4)

    #resample and map event to stream
    e_st = e_st3 + e_st4
    ce_st = e_st.copy() #keep a pre-resampled set for P-arrival picking (less memory)
    e_st.resample(1000)
    e_stream_dict[ekey] = e_st
    ce_stream_dict[ekey] = ce_st

#### 4.  Setup and test DATA directories and specfem3d file names

In [ ]:
experiments = []
for ekey in e2mt_keys:
    bkeys = dill_straces.getBoreholeKeys()
    bh_stat_list = []
    for bkey in bkeys:
        bh_stat_list.append(dill_straces.getIncludedStations(ekey,bkey))
    bh_stat_tup = tuple(bh_stat_list)   
    exp_set = {'EKEY':ekey,'EVENT':e_cat[ekey],'TENSOR':e2mt_dict[ekey],'STATIONS':bh_stat_tup}
    experiments.append(exp_set)

dir_set_list = []
min_eid = experiments[0]['EKEY']
for eset in experiments:
    eid = eset['EKEY']
    if eid < min_eid:
        min_eid = eid
        
dir_event_list = []
for eset in experiments:
    rdir = eset['EKEY'] - min_eid + 1
    rdir = 'run%s' %(str(rdir).zfill(4))
    dname = '../../../data_notebooks/SPECFEM3D/project/%s' %(rdir)
    dir_event_list.append((eset,dname))
    print('Experiment-%d:\n'%(eset['EKEY']), dname)
    print()

#### 5.  Create directories and write CMTSOLUTION, STATION, STATION_ADJOINT, and trace ascii files in separate dictories for each event

In [ ]:
from gnam.specutils.write import write_cmtsolution as spec_wcmt
from gnam.specutils.write import write_stations as spec_ws
import os

nproc = 16

# define the access rights
access_rights = 0o755

#loop over dir-set pairs
for pair in dir_event_list[::-1]:
    eset  = pair[0]
    dname = pair[1]
    new_dir = dname + '/DATA'
    print('Experiment-%d:\n'%(eset['EKEY']), dname)
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
    else:
        bkeys = dill_straces.getBoreholeKeys()
        lxy = dill_events.getLocalEventCoord(eset['EKEY'])
        spec_wcmt(new_dir + '/CMTSOLUTION',eset['TENSOR'],lxy[0],lxy[1])
        spec_ws(new_dir + '/STATIONS',dill_straces,eset['EKEY'],bkeys)
        spec_ws(new_dir + '/STATIONS_ADJOINT',dill_straces,eset['EKEY'],bkeys)
        
        #add links to bin and utils
        src = '/quanta1/home/tcullison/DevGPU_specfem3d/bin'
        dst = dname + '/bin'
        os.symlink(src, dst)
        src = '/quanta1/home/tcullison/DevGPU_specfem3d/utils'
        dst = dname + '/utils'
        os.symlink(src, dst)
        
    #add OUTPUT_FILES/DATABASES_MPI dirs
    new_dir  = dname + '/OUTPUT_FILES/DATABASES_MPI'
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
    else:
        if eset['EKEY'] != min_eid:
            #for k in ['Database','vp.bin','vs.bin','rho.bin']:
            for k in ['Database','external_mesh.bin']:
                for i in range(nproc):
                    iproc = str(i).zfill(6)
                    mesh_fname = 'proc%s_%s' %(iproc,k)
                    src = '../../../run0001/' + mesh_fname
                    dst = new_dir + '/' + mesh_fname
                    os.symlink(src, dst)
            mesh_fname = 'surface_from_mesher.h'
            src = '../../../run0001/OUTPUT_FILES/' + mesh_fname
            dst = dname + '/OUTPUT_FILES/' + mesh_fname
            os.symlink(src, dst)
            mesh_fname = 'values_from_mesher.h'
            src = '../../../run0001/OUTPUT_FILES/' + mesh_fname
            dst = dname + '/OUTPUT_FILES/' + mesh_fname
            os.symlink(src, dst)
    
    #add OBS dir
    new_dir  = dname + '/OBS'
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
    else:
        eid = eset['EKEY']
        e_st = e_stream_dict[eid]
        for tr in e_st:
            spec_pair_list = [] #time,amplitude pairs

            '''
            #Example of filename: 'NL.G094.FXX.semd'
            tr_filename = 'NL.' + tr.stats.station + '.FXZ.semd'
            print('filename:',tr_filename)

            for it in range(tr.count()):
                spec_pair_list.append('%E   %E\n' %(tr.times()[it],tr.data[it]))

            fqpname = new_dir + '/' + tr_filename
            f = open(fqpname, 'w')
            f.writelines(spec_pair_list)
            f.close()
            '''
            #Example of filename: 'NL.G094.FXX.semd'
            tr_spec_chan = '.BOO.'
            comp_char = tr.stats.channel[2] 
            if comp_char == 'Z':
                tr_spec_chan = '.FXZ.'
            elif comp_char == 'E':
                tr_spec_chan = '.FXX.'
            elif comp_char == 'N':
                tr_spec_chan = '.FXY.'
            else:
                print('Uh-oh! Spaghetti Os!')

            tr_filename = 'NL.' + tr.stats.station + tr_spec_chan + 'semd'

            for it in range(tr.count()):
                spec_pair_list.append('%E   %E\n' %(tr.times()[it],tr.data[it]))

            fqpname = new_dir + '/' + tr_filename
            f = open(fqpname, 'w')
            f.writelines(spec_pair_list)
            f.close()
        
        
    #add FILT_OBS dir
    new_dir  = dname + '/FILT_OBS'
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
        
        
    #add SYN dir
    new_dir  = dname + '/SYN'
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
        
        
    #add FILT_SYN dir
    new_dir  = dname + '/FILT_SYN'
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
        
        
    #add SEM dir
    new_dir  = dname + '/SEM'
    try:
        os.makedirs(new_dir, access_rights)
    except OSError:
        print ("Creation of the directory %s failed" % new_dir)
        print()
        
        
    #if run0001 dir (this is were the model setup and update happens)
    if eset['EKEY'] == min_eid:
        
        #add INPUT_KERNELS dir
        new_dir  = dname + '/INPUT_KERNELS'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        
        #add INPUT_MODEL dir
        new_dir  = dname + '/INPUT_MODEL'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        
        #add OUTPUT_MODEL dir
        new_dir  = dname + '/OUTPUT_MODEL'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        
        #add OUTPUT_SUM dir
        new_dir  = dname + '/OUTPUT_SUM'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        
        #add SMOOTH dir
        new_dir  = dname + '/SMOOTH'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        else:
            for k in ['alpha','beta','rho','hess']:
                for i in range(nproc):
                    iproc = str(i).zfill(6)
                    kernel_fname = 'proc%s_%s_kernel' %(iproc,k)
                    src = './' + kernel_fname + '_smooth.bin'
                    dst = new_dir + '/' + kernel_fname + '.bin'
                    os.symlink(src, dst)
        
        #add INPUT_GRADIENT dir
        new_dir  = dname + '/INPUT_GRADIENT'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        else:
            for k in ['alpha','beta','rho','hess']:
                for i in range(nproc):
                    iproc = str(i).zfill(6)
                    kernel_fname = 'proc%s_%s_kernel_smooth' %(iproc,k)
                    src = '../SMOOTH/' + kernel_fname
                    dst = new_dir + '/' + kernel_fname + '.bin'
                    os.symlink(src, dst)
            
        #add topo dir
        new_dir  = dname + '/topo'
        try:
            os.makedirs(new_dir, access_rights)
        except OSError:
            print ("Creation of the directory %s failed" % new_dir)
            print()
        else:
            for i in range(nproc):
                iproc = str(i).zfill(6)
                ext_mesh_fname = 'proc%s_external_mesh.bin' %(iproc)
                src = '../OUTPUT_FILES/DATABASES_MPI/' + ext_mesh_fname
                dst = new_dir + '/' + ext_mesh_fname
                os.symlink(src, dst)


#### 6. Pick P-arrivals

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from obspy.core import read
from obspy.signal.trigger import ar_pick


czne_st = ce_st.copy()

df = czne_st[0].stats.sampling_rate
f1 = 3.0
f2 = 12.0
lta_p = 1.0
sta_p = 0.1
lta_s = 4.0
sta_s = 1.0
m_p = 2
m_s = 8
l_p = 0.1
l_s = 0.2

p_sfudge = 0.75
s_efudge = 0.75

czne_st[0:3].filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)

p_pick, s_pick = ar_pick(czne_st[0].data, czne_st[1].data, czne_st[2].data, df,
                         f1, f2, lta_p, sta_p, lta_s, sta_s, m_p, m_s, l_p, l_s)

#print(p_pick)
#print(s_pick)

fig, ax = plt.subplots(1,figsize=(14,5))
#print('czne_st[0]:\n',vars(czne_st[0]))
z_data = czne_st[0].data
n_data = czne_st[1].data
e_data = czne_st[2].data
dt     = czne_st[0].stats.delta
nt     = czne_st[0].count()
t      = czne_st[0].times()
#print('dt:',dt)
#print('nt:',nt)
#print('df:',df)
ax.plot(t, z_data,c='black')
ax.plot(t, n_data,c='green')
ax.plot(t, e_data,c='orange')
amin_z = np.min(z_data)
amax_z = np.max(z_data)
amin_n = np.min(n_data)
amax_n = np.max(n_data)
amin_e = np.min(e_data)
amax_e = np.max(e_data)
amin = np.min([amin_z,amin_n,amin_e])*2
amax = np.max([amax_z,amax_n,amax_e])*2
#print('amax:',amax)
#print('amin:',amin)
ax.vlines(x=p_pick, ymin=amin, ymax=amax, colors='red',linestyle='dashed')
ax.vlines(x=p_pick-p_sfudge, ymin=amin, ymax=amax, colors='red')
ax.vlines(x=s_pick, ymin=amin, ymax=amax, colors='blue',linestyle='dashed')
ax.vlines(x=s_pick+s_efudge, ymin=amin, ymax=amax, colors='blue')

plt.grid(b=True, which='major', color='#666666', linestyle='-')
plt.minorticks_on()
plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)

plt.show()

In [ ]:
import numpy.random as rand
#rand.seed(2)
cekeys = ce_stream_dict.keys()
min_cekey = min(cekeys)
max_cekey = max(cekeys)
print('min key:',min_cekey)
print('max key:',max_cekey)
rand_cekey = rand.randint(min_cekey, max_cekey)
rand_cekey = min_cekey+2
print(rand_cekey)

In [ ]:
import obspy
from obspy import Stream
from obspy import Trace
from obspy import UTCDateTime
import matplotlib.pyplot as plt
import numpy as np
import copy 
from obspy.signal.trigger import recursive_sta_lta
from obspy.signal.trigger import plot_trigger

dc_real = copy.deepcopy(ce_stream_dict[rand_cekey])
df = ce_st[0].stats.sampling_rate
f1 = 3.0
f2 = 18.0
lta_p = 1.5
sta_p = 0.5
lta_s = 2
sta_s = 0.5
m_p = 8
m_s = 8
l_p = 0.5
l_s = 0.5

p_trig_pad = 0.75
s_trig_pad = 0.75

dc_real.filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)

plt_h = 2
plt_w = 14
ntrace = len(dc_real)//3
plt_scale = int(1*len(dc_real[:3*ntrace:3]))
fig, ax = plt.subplots(plt_scale,figsize=(plt_w,plt_h*plt_scale))
fig.tight_layout()
zshft = int(0)
nshft = int(1)
eshft = int(2)
shft = zshft
triggers = np.zeros((ntrace,4))
trig_dict = {}
for i in range(len(dc_real[:3*ntrace:3])):
    
    #dc_real[shft+i*3].filter('bandpass',freqmin=f1,freqmax=f2,corners=4,zerophase=True)
    
    #real_data = dc_real[shft+i*3].data
    
    dt     = dc_real[i*3].stats.delta
    nt     = dc_real[i*3].count()
    t      = dc_real[i*3].times()
    
    p_0, s_0 = ar_pick(dc_real[i*3].data, dc_real[i*3+1].data, dc_real[i*3+2].data, df, 
                       f1, f2, lta_p, sta_p, lta_s, sta_s, m_p, m_s, l_p, l_s)
    
    if s_0 < p_0:
        s_0 = 2*p_0
        
    #s_0 = 2*p_0
    #p_0 = 0.5*s_0
    
    # leave 0.5 sec for tapper -- this is just a guess
    p_trig = p_0 - p_trig_pad
    if p_trig < 0.5:
        p_trig = 0.5
    s_trig = s_0 + s_trig_pad
    if nt*dt - 0.5 <= s_trig:
        s_trig = nt*dt - 0.5
        
    triggers[i,0] = p_trig
    triggers[i,1] = p_0
    triggers[i,2] = s_0
    triggers[i,3] = s_trig
    trig_dict[dc_real[shft+i*3].stats.station] = np.array(triggers[i,:])
    
    ax[i].plot(t, dc_real[i*3].data,c='black',zorder=2)
    ax[i].plot(t, dc_real[i*3+1],c='green',zorder=1)
    ax[i].plot(t, dc_real[i*3+2],c='orange',zorder=0)
    
    amin = np.min(dc_real[i*3].data)
    amin = np.min(np.array([amin,np.min(dc_real[i*3+1].data)]))
    amin = np.min(np.array([amin,np.min(dc_real[i*3+2].data)]))
    amax = np.max(dc_real[i*3].data)
    amax = np.max(np.array([amax,np.max(dc_real[i*3+1].data)]))
    amax = np.max(np.array([amax,np.max(dc_real[i*3+2].data)]))
    ax[i].vlines(x=p_0, ymin=amin, ymax=amax, colors='red',linestyle='dashed')
    ax[i].vlines(x=p_trig, ymin=amin, ymax=amax, colors='red')
    ax[i].vlines(x=s_0, ymin=amin, ymax=amax, colors='blue',linestyle='dashed')
    ax[i].vlines(x=s_trig, ymin=amin, ymax=amax, colors='blue')
    
    rstation = dc_real[i*3].stats.station
    rstation += '.' + dc_real[shft+i*3].stats.channel
    
    overlay_title = 'Station:' + str(rstation)
    ax[i].set_title(overlay_title)
    
    ax[i].grid(b=True, which='major', color='#666666', linestyle='-')
    ax[i].minorticks_on()
    ax[i].grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
plt.show()


#### 7. Write adjoint source window parameter files 

## Finished